In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from pyclustering.cluster.kmedoids import kmedoids
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from umap import UMAP
from sklearn.preprocessing import StandardScaler
import prince

import gower

import plotly.express as px
from ipywidgets import interact

%matplotlib inline
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

<IPython.core.display.Javascript object>

In [3]:
# dask imports
import dask
import dask.dataframe as dd
import joblib

<IPython.core.display.Javascript object>

In [4]:
# setting up dask
import warnings

warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit="2GB")
client

Client Scheduler: tcp://127.0.0.1:55963 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


<IPython.core.display.Javascript object>

### Load in and Prepare Data

Data is cleaned and explored in accompanying notebook. If downloaded from github, data files will need to be unzipped.

In [8]:
loans = dd.read_csv("data/SBA_clnd.csv")
loans.head()

,Name,City,State,Zip,Bank,BankState,ApprovalFY,NoEmp,NewExist,CreateJob,RetainedJob,RevLineCr,LowDoc,DisbursementGross,MIS_Status,twoDigNAICS,is_franchise,bank_out_of_state,Term_years,UrbanRural_cleaned,Disbr_year,Disbr_Month_sin,Disbr_Month_cos,sba_pre_approv,percent_SBA,bank_size,Appv_Month_sin,Appv_Month_cos
0,CARVEL,APEX,NC,27502,STEARNS BK NATL ASSOC,MN,2006,2,1,0,0,0.0,0.0,253400.0,0,44,1.0,0,13.500000,0.0,2006,0.866025,5.000000e-01,1.0,0.75,1.0,0.5,0.866025
1,SUBWAY,LITTLE ROCK,AR,72223,HOPE FCU,MS,2006,7,0,0,0,0.0,0.0,137300.0,0,72,0.0,0,10.500000,0.0,2006,1.000000,6.123234e-17,1.0,0.85,0.0,0.5,0.866025
2,WEYLAND CORPORATION,CAMARILLO,CA,93010,WELLS FARGO BANK NATL ASSOC,SD,2006,18,1,5,23,1.0,0.0,438541.0,0,61,0.0,0,6.916667,0.0,2006,0.500000,8.660254e-01,1.0,0.50,2.0,0.5,0.866025
3,CHICAGO BRICK UNLIMITED INC,MIAMI,FL,33186,"CITIBANK, N.A.",FL,2006,4,0,0,4,1.0,0.0,51440.0,0,23,0.0,1,7.000000,0.0,2006,0.500000,8.660254e-01,1.0,0.50,2.0,0.5,0.866025
4,"RZI, INC.",NEW ORLEANS,LA,70130,BUSINESS RES. CAP. SPECIALTY B,LA,2006,3,0,0,0,0.0,0.0,50000.0,0,53,0.0,1,5.000000,0.0,2006,0.866025,-5.000000e-01,1.0,0.85,0.0,0.5,0.866025


<IPython.core.display.Javascript object>

Create a column to represent the size of a bank by the number of loans given over this period

In [16]:
loans["Bank"].value_counts()

Dask Series Structure:
npartitions=1
    int64
      ...
Name: Bank, dtype: int64
Dask Name: value-counts-agg, 7 tasks

<IPython.core.display.Javascript object>

In [19]:
client.close()

<IPython.core.display.Javascript object>

In [17]:
bank_counts = loans["Bank"].value_counts().compute()
loans["bank_size"] = loans["Bank"].apply(lambda x: bank_counts[x])

ValueError: cannot reindex from a duplicate axis

<IPython.core.display.Javascript object>

Select defaulted loans only.

In [ ]:
default = loans[loans["MIS_Status"] == 1]
#default = default[default["NoEmp"] <= 100]


Drop columns not useful for clustering

In [ ]:
drop_cols = [
    "Name",
    "City",
    "Zip",
    "Bank",
    "ApprovalFY",
    "sba_pre_approv",
    "Disbr_Month_sin",
    "Disbr_Month_cos",
    "Appv_Month_sin",
    "Appv_Month_cos",
    "MIS_Status",
    "twoDigNAICS",
    "State",
    "BankState",
]
default = default.drop(columns=drop_cols)

Categorize columns into numerica and categorical for preprocessing

In [ ]:
num_cols = [
    "NoEmp",
    "CreateJob",
    "RetainedJob",
    "DisbursementGross",
    "Term_years",
    "Disbr_year",
    "percent_SBA",
    "bank_size",
]

cat_cols = [
    "NewExist",
    "RevLineCr",
    "LowDoc",
    "is_franchise",
    "bank_out_of_state",
    "UrbanRural_cleaned",
]

# will want cat_cols as str for discrete coloring in graphs
# converting here breaks famd
# loans[cat_cols] = loans[cat_cols].astype("str")
is_cat = default.columns.isin(cat_cols)

In [ ]:
default.head()

Sample 1,000 loans so distance matrixes and clustering algorithms run in a reasonable amount of time.

In [ ]:
sample = default.sample(1000, random_state = 66)

# will want cat_cols as str for discrete coloring in graphs
sample[cat_cols] = sample[cat_cols].astype("str")
sample.info()


### Calculate Gower Distance Matrix for Mixed Data

In [ ]:
gower_dist = gower.gower_matrix(sample, cat_features=is_cat)
gower_df = pd.DataFrame(gower_dist, columns=sample.index, index=sample.index)

Scale the numerical features for clustering to give features equal voices

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(sample[num_cols])
X_std = pd.DataFrame(X_std, columns=num_cols, index=sample[num_cols].index)
# pd.DataFrame(scaler.fit_transform(X[num_cols]), columns=num_cols)

scaled_X = pd.concat((X_std, sample[cat_cols]), axis=1)

# for after cluster, doing some preliminary visualization
# scaled_X["label"] = labels
scaled_X

In [ ]:
scaled_X.describe()

### Dimension Reduction

Factor Analysis of Mixed Data is ran to visualize the sample. Most of the variation appears to be on one axis.

In [ ]:
famd = prince.FAMD(n_components=2)
famd.fit(scaled_X)

In [ ]:
famd.plot_row_coordinates(
    scaled_X, alpha=0.5, ellipse_fill=False,
)
plt.show()

In [ ]:
# store the famd coordinates to do custom plotting
sample["famd_x"] = famd.row_coordinates(scaled_X).iloc[:, 0]
sample["famd_y"] = famd.row_coordinates(scaled_X).iloc[:, 1]

Just from the Factor Analysis, it appears there are at least two dense clusters, with some outliers. DBSCAN should perform excellently here.

* Note: DBSCAN did not perform well due to sparse, subtle clusters. K-medoids was chosen to counter skew, however transforming the data may result in different cluster separations.

### K-Medoids

In [ ]:
np.random.seed(613)
k = 3
nrows = gower_df.shape[0]
init_medoids = np.random.randint(0, nrows, k)
init_medoids

In [ ]:
kmed = kmedoids(
    gower_dist, initial_index_medoids=init_medoids, data_type="distance_matrix",
)

kmed.process()

In [ ]:
labels = kmed.predict(gower_dist)
sample_kmeds = sample.copy()
sample_kmeds["label"] = labels
sample_kmeds["label"] = sample_kmeds["label"].astype("str")
sample_kmeds["label"].value_counts(normalize=True)

In [ ]:
num_cols_labels = num_cols.copy()
num_cols_labels.append("label")
clst_avg = sample_kmeds[num_cols_labels].groupby("label").mean().T
clst_avg.style.background_gradient(axis=1)

In [ ]:
cat_cols_labels = cat_cols.copy()
cat_cols_labels.append("label")
clst_avg = sample_kmeds[cat_cols_labels].groupby("label").agg(pd.Series.mode).T
clst_avg.style.background_gradient(axis=1)

In [ ]:
@interact
def plot(color=sample_kmeds.columns, opacity=(0, 1.0)):
    fig = px.scatter(
        data_frame=sample_kmeds,
        x="famd_x",
        y="famd_y",
        color=color,
        hover_data=["DisbursementGross", "LowDoc", "label"],
        # symbol="label",
    )
    fig.update_traces(marker={"opacity": opacity})
    fig.show()

### DBSCAN

In [ ]:
clst = DBSCAN(eps=0.1, min_samples=10, metric="precomputed")
clst.fit(gower_df)

In [ ]:
sample_dbscan = sample.copy()
sample_dbscan["label"] = clst.labels_
sample_dbscan["label"] = sample_dbscan["label"].astype("str")
sample_dbscan["label"].value_counts()

In [ ]:
num_cols_labels = num_cols.copy()
num_cols_labels.append("label")
clst_avg = sample_dbscan[num_cols_labels].groupby("label").mean().T
clst_avg.style.background_gradient(axis=1)

In [ ]:
cat_cols_labels = cat_cols.copy()
cat_cols_labels.append("label")
clst_avg = sample_dbscan[cat_cols_labels].groupby("label").agg(pd.Series.mode).T
clst_avg.style.background_gradient(axis=1)

In [ ]:
# for dbscan, get rid of outliers
sample_dbscan = sample_dbscan[sample_dbscan["label"] != "-1"]


@interact
def plot(color=sample_dbscan.columns, opacity=(0, 1.0)):
    fig = px.scatter(
        data_frame=sample_dbscan,
        x="famd_x",
        y="famd_y",
        color=color,
        hover_data=["DisbursementGross", "LowDoc", "label"],
        # symbol="label",
    )
    fig.update_traces(marker={"opacity": opacity})
    fig.show()

### Hierarchical

In [ ]:
clst = AgglomerativeClustering(n_clusters=4, affinity="precomputed", linkage="complete")
clst.fit(gower_df)

In [ ]:
sample_hier = sample.copy()
sample_hier["label"] = clst.labels_
sample_hier["label"] = sample_hier["label"].astype("str")
sample_hier["label"].value_counts()

In [ ]:
num_cols_labels = num_cols.copy()
num_cols_labels.append("label")
clst_avg = sample_hier[num_cols_labels].groupby("label").mean().T
clst_avg.style.background_gradient(axis=1)

In [ ]:
cat_cols_labels = cat_cols.copy()
cat_cols_labels.append("label")
clst_avg = sample_hier[cat_cols_labels].groupby("label").agg(pd.Series.mode).T
clst_avg.style.background_gradient(axis=1)

In [ ]:
@interact
def plot(color=sample_hier.columns, opacity=(0, 1.0)):
    fig = px.scatter(
        data_frame=sample_hier,
        x="famd_x",
        y="famd_y",
        color=color,
        hover_data=["DisbursementGross", "LowDoc", "label"],
        # symbol="label",
    )
    fig.update_traces(marker={"opacity": opacity})
    fig.show()

### Clustering Notes

DBSCAN tended to create a large outliers clusters, or too many small clusters. This is likely due to the sparseness of the data. Some of these clusters may be informative, but may not represent larger trends. Heirarchical clustering also had similar results, with a large majority class and remaining clusters orders of magnitude smaller. K-Medoids was chosen as the best algorithm and as way to address skew, since median measures are more resilient to outliers. Three clusters were chosen as larger values of K seemed to result in 2 consistent, distinct groups with the remaining clusters appearing similar to what is group 2 (index starting at 0) in the final clustering.

### Cluster Analysis Visualization

In [ ]:
sample_kmeds["label"].value_counts()

In [ ]:
# code to convert counts to percents for cluster comparison

sample_kmeds["bank_out_of_state"] = sample_kmeds["bank_out_of_state"].astype("int")
out_of_state_percs = pd.DataFrame(
    sample_kmeds.groupby("label")["bank_out_of_state"].agg("mean"),
)
sns.barplot(x=out_of_state_percs.index, y="bank_out_of_state", data=out_of_state_percs)

sample_kmeds["NewExist"] = sample_kmeds["NewExist"].astype("int")
new_exist_percs = pd.DataFrame(
    sample_kmeds.groupby("label")["NewExist"].agg("mean"),
)
sns.barplot(x=new_exist_percs.index, y="NewExist", data=new_exist_percs)
plt.close()


In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 8), sharex=True)
sns.barplot(ax=axs[0, 0], x="label", y="DisbursementGross", data=sample_kmeds)
axs[0, 0].set_title("Avg. Loan Amount ($)")
axs[0, 0].set_ylabel("")

sns.barplot(ax=axs[0, 1], x=new_exist_percs.index, y="NewExist", data=new_exist_percs)
# sns.countplot( x="label", hue="NewExist", data=sample_kmeds)
axs[0, 1].set_title("New Businesses (Percent)")
# axs[0, 1].legend(["No", "Yes"], loc="upper right")
axs[0, 1].set_ylabel("")

sns.barplot(
    ax=axs[0, 2],
    x=out_of_state_percs.index,
    y="bank_out_of_state",
    data=out_of_state_percs,
)
# sns.countplot( x="label", hue="bank_out_of_state", data=sample_kmeds)
axs[0, 2].set_title("Out of State Bank (Percent)")
axs[0, 2].set_ylabel("")

sns.barplot(ax=axs[1, 0], x="label", y="Term_years", data=sample_kmeds)
axs[1, 0].set_title("Loan Term (Years)")
axs[1, 0].set_ylabel("")

sns.barplot(ax=axs[1, 2], x="label", y="bank_size", data=sample_kmeds)
axs[1, 2].set_title("Bank Size (Loans Given)")
axs[1, 2].set_ylabel("")

sns.barplot(ax=axs[1, 1], x="label", y="CreateJob", data=sample_kmeds)
axs[1, 1].set_title("Jobs Created (Count)")
axs[1, 1].set_ylabel("")

# fig.savefig("clusters.png")

plt.show()

### Cluster Interpretations

Cluster 0: Failure to Launch

Features:
* Highest Percentage of New Businesses
Biggest Loans, Longest Average Loan Term
* Smallest Banks, but most out-of-state
* Created the most jobs

Interpretation: Promising new businesses that faltered

Suggested Actions:
* Consider for a second chance
* May need guidance and adjustments
* Connect with more conservative loans
<hr/>

Cluster 1: Business Burnout

Features:
* High percent of In-State Banks, Biggest Banks
* Highest percent of Existing Businesses
* Shortest Average Loan Term
* Created the Least Jobs
* Biggest Cluster (~500 loans)

Interpretation: Businesses with exhausted local credit options and slowing growth

Suggested Actions:
* Connect with new resources, out-of-state banks
* Help modernize
<hr/>

Cluster 2: Average Janes 

Features:
* Similar businesses to Group 2
* In-between loan sizes and terms
* Medium sized, in-state banks

Interpretation: Businesses on their way to burning out

Suggested Actions:
* Connect with new resources, bigger banks
* May have features of either first 2 groups
* Help overcome the obstacles that result in Cluster 2
* Build on the success that kep them from becoming Cluster 1




### Suggested Actions

* New businesses and established businesses have different needs

* New Businesses:
    * Marketing, Operations, and Vision
    * Help get a foothold

* Established Businesses:
    * Assist with upscaling
    * Connect with contractors for new responsibilities

* Advise business on credit opportunities based on what already been utilized


### Limitations

* Interpretation of Bank Size and State
    * Some businesses may go out of state for lower rates
    * Interpreted here as struggling to find credit
* Skewed Data
    * Data transformations can help 
* SBA definition of atypical loans for fringe cases
* Would like to know if these businesses have sought SBA loans before
    * Also business age, not just new or existing


### Further Analysis

* Internal SBA classifications can inform cluster interpretation
    * Did the clusters challenge trends, or describe known patterns?
* More data engineering
    * Refine Bank Size feature
     * Effect of Industry, State Economy, and Year 
* A/B testing to see if suggested further assistance is effective


Some EDA

In [ ]:
sns.distplot(default["DisbursementGross"])
plt.title("Loan Amount")
plt.xlabel("Dollars")
# plt.savefig("loan_amount.png")
plt.show()

In [ ]:
default["DisbursementGross"].describe()

In [ ]:
sns.distplot(default["Term_years"])
plt.title("Loan Term")
plt.xlabel("Years")
plt.ylabel("Frequency")
# plt.savefig("loan_term.png")
plt.show()

In [ ]:
default["Term_years"].describe()

In [ ]:
default["Term_years"].value_counts().sort_index()

In [ ]:
sns.boxplot(default["bank_size"])
plt.xlabel("Number of Loans")
plt.title("Bank Size")
# plt.savefig("bank_size.png")
plt.show()

In [ ]:
default["bank_size"].describe()

In [ ]:
default["bank_size"].value_counts().sort_index()

In [ ]:
sns.boxplot(np.log(default["NoEmp"]))
plt.xlabel("Log of Number of Employees")
plt.ylabel("Frequency")
plt.show()

In [ ]:
default["NoEmp"].value_counts().sort_index()

In [ ]:
default["NoEmp"].describe()

In [ ]:
sns.distplot(default["CreateJob"])
plt.xlabel("Jobs Created")
plt.show()

In [ ]:
default["CreateJob"].value_counts().sort_index()

In [ ]:
sns.countplot(x="NewExist", data=default)
plt.xlabel("Bank Out Of State")
plt.show()

In [ ]:
default["NewExist"].value_counts(normalize=True)

In [ ]:
sns.countplot(x="bank_out_of_state", data=default)
plt.xlabel("Bank Out Of State")
plt.show()

In [ ]:
default["bank_out_of_state"].value_counts(normalize=True)

In [ ]:
# long out put
# default.groupby("State")["bank_out_of_state"].value_counts(normalize=True)

In [ ]:
sns.distplot(sample["NoEmp"])

In [ ]:
sns.distplot(sample["CreateJob"])